In [1]:
import os

print(os.getcwd())
os.chdir("../")
print(os.getcwd())

/home/mr-rey/Joseph/Projects/Python/coccidiosis-detection-ai/research
/home/mr-rey/Joseph/Projects/Python/coccidiosis-detection-ai


In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [3]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH ):
        # schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        # self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])   
        
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(config.tensorboard_root_log_dir),
            Path(model_ckpt_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [5]:
import os
import urllib.request as request
import zipfile
import tensorflow as tf
import time

2025-07-08 13:08:58.932574: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-08 13:08:59.000959: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-08 13:08:59.286162: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-08 13:09:00.498866: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [8]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    print(callback_list)
except Exception as e:
    raise e

[2025-07-08 13:12:37,274: INFO: common]: yaml file: config/config.yaml loaded successfully
[2025-07-08 13:12:37,275: INFO: common]: yaml file: params.yaml loaded successfully
[2025-07-08 13:12:37,276: INFO: common]: created directory at: artifacts
[2025-07-08 13:12:37,276: INFO: common]: created directory at: artifacts/prepare_callbacks/tensorboard_log_dir
[2025-07-08 13:12:37,277: INFO: common]: created directory at: artifacts/prepare_callbacks
[<keras.src.callbacks.tensorboard.TensorBoard object at 0x730e91bc1090>, <keras.src.callbacks.model_checkpoint.ModelCheckpoint object at 0x730e91bc8290>]
